In [ ]:
!pip install fluidsynth

  Created wheel for fluidsynth: filename=fluidsynth-0.2-py3-none-any.whl size=4513 sha256=fd2913fb190ae5abb61d82ed6bea5c5dc2529d786e45f3b5c7cb3694f34973a0
  Stored in directory: /root/.cache/pip/wheels/44/4b/81/dee2d535b2dd27af13c230ba49aa6f05a5dff0f57c68acef55
Successfully built fluidsynth


In [ ]:
!apt install fluidsynth

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 37 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt5x11extras5 amd64 5.9.5-0ubuntu1 [8,596 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic/uni

In [ ]:
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

In [ ]:
path = './drive/MyDrive/Midi_Project/'

In [ ]:
#defining function to read MIDI files
def read_midi(file):
    
    print("Loading Music File:",file)
    
    notes = np.load(path+file, allow_pickle=True)

    for i in range(len(notes)):
      notes[i]= np.array(notes[i])

    return np.array(notes)

In [ ]:
#for listing down the file names
import os

#Array Processing
import numpy as np

#reading each midi file
notes_array = read_midi("data/train_bpe.npy")

Loading Music File: data/train_bpe.npy


In [ ]:
!pip install tokenizers

     |████████████████████████████████| 6.8 MB 5.0 MB/s 


In [ ]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_file(path + "data/vocab_bpe.json")

In [ ]:
unique_notes = tokenizer.get_vocab_size()
#unique_notes=300

In [ ]:
unique_notes

300

In [ ]:
len(notes_array)

45848

In [ ]:
time_steps = 32
x = []
y = []

for notes in notes_array[:1000]:
  for i in range(0, len(notes)-time_steps):
    input = notes[i:i+time_steps]
    output = notes[i+time_steps]
    x.append(input)
    y.append(output)

new_x = np.array(x)
new_y = np.array(y)

In [ ]:
del x

In [ ]:
del y

In [ ]:
len(new_x)

337166

In [ ]:
from sklearn.model_selection import train_test_split as tts
x_train, x_val, y_train, y_val = tts(new_x, new_y, test_size = 0.2, random_state=0)

In [ ]:
def LSTM_model(n_vocab):
  model = keras.models.Sequential()
  model.add(keras.layers.Embedding(unique_notes, 100, input_length=time_steps,trainable=True))
  model.add(keras.layers.LSTM(256, return_sequences=True))
  model.add(keras.layers.Dropout(0.25))
  model.add(keras.layers.LSTM(512, return_sequences=True))
  model.add(keras.layers.Dropout(0.25))
  model.add(keras.layers.LSTM(256))
  model.add(keras.layers.Dense(256, activation='relu'))
  model.add(keras.layers.Dropout(0.25))
  model.add(keras.layers.Dense(unique_notes, activation='softmax'))
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = 'Adam')
  return model

In [ ]:
model = LSTM_model(unique_notes)

In [ ]:
mc = keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)

In [ ]:
history = model.fit(np.array(x_train),np.array(y_train),batch_size=128,epochs=20, validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

Epoch 1/20
2107/2108 [============================>.] - ETA: 0s - loss: 3.0957
Epoch 00001: val_loss improved from inf to 1.88902, saving model to best_model.h5
2108/2108 [==============================] - 172s 78ms/step - loss: 3.0955 - val_loss: 1.8890
Epoch 2/20
2107/2108 [============================>.] - ETA: 0s - loss: 1.7653
Epoch 00002: val_loss improved from 1.88902 to 1.57904, saving model to best_model.h5
2108/2108 [==============================] - 164s 78ms/step - loss: 1.7652 - val_loss: 1.5790
Epoch 3/20
2107/2108 [============================>.] - ETA: 0s - loss: 1.5791
Epoch 00003: val_loss improved from 1.57904 to 1.48717, saving model to best_model.h5
2108/2108 [==============================] - 160s 76ms/step - loss: 1.5791 - val_loss: 1.4872
Epoch 4/20
2107/2108 [============================>.] - ETA: 0s - loss: 1.4903
Epoch 00004: val_loss improved from 1.48717 to 1.41365, saving model to best_model.h5
2108/2108 [==============================] - 161s 76ms/step - 

In [ ]:
from keras.models import load_model
model = load_model(path+'models/best_model_bpe.h5')

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
time_steps=32

In [ ]:
def temperature_sampling(logits, temperature=1, topk=1):
        probs = np.exp(logits / temperature) / np.sum(np.exp(logits / temperature))
        if topk == 1:
            prediction = np.argmax(probs)
        else:
            sorted_index = np.argsort(probs)[::-1]
            candi_index = sorted_index[:topk]
            candi_probs = [probs[i] for i in candi_index]
            # normalize probs
            candi_probs /= sum(candi_probs)
            # choose by predicted probs
            prediction = np.random.choice(candi_index, size=1, p=candi_probs)[0]
        return prediction

def generate_music(Notes_len=200):
    ind = np.random.randint(0,1000-1)
    #ind = 826
    #ind=543
    #ind=789
    ind+=1000
    random_music = notes_array[ind][:time_steps]

    predictions=list(random_music)
    for i in tqdm(range(Notes_len)):

        random_music = random_music.reshape(1,time_steps)

        prob  = model.predict(random_music)[0]
        y_pred = temperature_sampling(prob, temperature=1, topk=1)
        predictions.append(y_pred)

        random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
        random_music = random_music[1:]
    
    return predictions
    


In [ ]:
!pip install miditok

     |████████████████████████████████| 58 kB 2.9 MB/s 
     |████████████████████████████████| 15.7 MB 11.9 MB/s 
     |████████████████████████████████| 51 kB 6.8 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!pip install miditoolkit

In [ ]:
tokenizer_nlp = Tokenizer.from_file(path + "data/vocab_bpe.json")

In [ ]:
from miditok import REMIEncoding, get_midi_programs
from miditoolkit import MidiFile

In [ ]:
text = '[CLS]'
pitch_range = range(21, 109)
beat_res = {(0, 4): 8, (4, 12): 4}
nb_velocities = 32
additional_tokens = {'Chord': False, 'Rest': True, 'Tempo': False, 'Program': False,
                     'rest_range': (2, 8),  # (half, 8 beats)
                     'nb_tempos': 32,  # nb of tempo bins
                     'tempo_range': (40, 250)}  # (min, max)
tokenizer_midi = REMIEncoding(pitch_range, beat_res, nb_velocities, additional_tokens)

miditok: REMIEncoding class has been renamed REMI and will be removed in future updates, please consider changing it in your code


In [ ]:
def convert_to_midi(predicted_sequences, file_save="sample.mid"):
  s1 = tokenizer_nlp.decode(predicted_sequences)
  s1 = "".join(s1.split(' '))
  tokens_real = []
  for i in range(len(s1)):
      tokens_real.append(int(ord(s1[i])-int(0x2200)))
  print("Predicted Sequence Length: ", len(predicted_sequences))
  print("Midi token length Remi: ", len(tokens_real))
  
  gen_midi = tokenizer_midi.tokens_to_midi([tokens_real], [(0, False)])
  gen_midi.dump(file_save)
  print("Midi saved successfully")
  print()

In [ ]:
folder = "Songs/bpe/"
import os

In [ ]:
for i in range(20):
  save_name = path + folder + "sample_" + str(i+1) + "_500.mid"
  save_wav = path + folder + "sample_"+str(i+1)+"_500.wav"
  predicted_sequences = generate_music(Notes_len=500)
  convert_to_midi(predicted_sequences, file_save=save_name)
  os.system("fluidsynth -ni font.sf2 %s -F %s -r 44100"%(save_name, save_wav))

  0%|          | 0/500 [00:00<?, ?it/s]

Predicted Sequence Length:  532
Midi token length Remi:  1324
./drive/MyDrive/Midi_Project/Songs/bpe/sample_1_500.mid
Midi saved successfully



  0%|          | 0/500 [00:00<?, ?it/s]

Predicted Sequence Length:  532
Midi token length Remi:  1348
./drive/MyDrive/Midi_Project/Songs/bpe/sample_2_500.mid
Midi saved successfully



  0%|          | 0/500 [00:00<?, ?it/s]

...

Predicted Sequence Length:  532
Midi token length Remi:  1295
./drive/MyDrive/Midi_Project/Songs/bpe/sample_18_500.mid
Midi saved successfully



  0%|          | 0/500 [00:00<?, ?it/s]

Predicted Sequence Length:  532
Midi token length Remi:  1087
./drive/MyDrive/Midi_Project/Songs/bpe/sample_19_500.mid
Midi saved successfully



  0%|          | 0/500 [00:00<?, ?it/s]

Predicted Sequence Length:  532
Midi token length Remi:  1697
./drive/MyDrive/Midi_Project/Songs/bpe/sample_20_500.mid
Midi saved successfully



In [ ]:
for i in range(5):
  save_name = path + folder + "sample_" + str(i) + ".mid"
  predicted_sequences = generate_music(Notes_len=200)
  convert_to_midi(predicted_seq)

In [ ]:
folder = "samples_bpe/"

# Perplexity Evaluation

In [ ]:
samples = np.array([272, 397, 524, 711, 264, 469, 920, 598, 601, 962, 774, 381, 608,
       592,  61, 305, 234, 579, 692, 894,  15, 925, 397, 298,  14, 476,
       412, 791,  77, 528, 322, 115, 825, 446, 256, 568, 173, 321, 155,
       182, 685, 395, 870, 394, 282, 718, 730, 548, 573, 733, 238, 581,
       712, 758, 986, 467, 622, 865, 630, 745, 364, 957, 478, 846, 553,
       441, 991, 182, 308, 102, 766, 837, 765, 666, 253, 788, 935, 802,
       319, 333, 163, 371, 522, 699,  70, 816, 413, 570, 754,  99, 441,
       303, 701, 194, 643, 391, 543, 128, 643,  21])

In [ ]:
def temperature_sampling(logits, temperature=1, topk=1):
        probs = np.exp(logits / temperature) / np.sum(np.exp(logits / temperature))
        if topk == 1:
            prediction = np.argmax(probs), probs
        else:
            sorted_index = np.argsort(probs)[::-1]
            candi_index = sorted_index[:topk]
            candi_probs = [probs[i] for i in candi_index]
            # normalize probs
            candi_probs /= sum(candi_probs)
            # choose by predicted probs
            prediction = np.random.choice(candi_index, size=1, p=candi_probs)[0]
        return prediction

def perplexity(sample_size=100, eval_notes=32, samples=[]):
    ppx = 0
    for i in tqdm(range(sample_size)):
      ind = np.random.randint(0,1000-1)
      if len(samples)!=0:
        ind = samples[i]
      #ind = 826
      #ind=543
      #ind=789
      ind+=1000     
      prob_total =1.0;
      for i in range(eval_notes):
          random_music = notes_array[ind][i:time_steps+i]
          random_music = random_music.reshape(1,time_steps)

          prob  = model.predict(random_music)[0]
          y_pred, prob = temperature_sampling(prob, temperature=1, topk=1)
          prob_total*=prob[notes_array[ind][time_steps+i]]
      ppx+=1/np.power(prob_total,1/eval_notes)

    return ppx/sample_size
      
      


In [ ]:
perplexity(sample_size=50, samples=samples)

  0%|          | 0/50 [00:00<?, ?it/s]

194.60723508372342

In [ ]:
predicted_sequences = generate_music(Notes_len=500)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
s1 = tokenizer.decode(predicted_sequences)
s1 = "".join(s1.split(' '))
tokens_real = []
for i in range(len(s1)):
    tokens_real.append(int(ord(s1[i])-int(0x2200)))

In [ ]:
len(predicted_sequences)

532

In [ ]:
len(tokens_real)

1247

In [ ]:
save_name = path + "samples_bpe/" + "sample_4_500.mid"
gen_midi = tokenizer_midi.tokens_to_midi([tokens_real], [(0, False)])
gen_midi.dump(save_name)

./drive/MyDrive/Midi_Project/samples_bpe/sample_4_500.mid


In [ ]:
ind = 851
random_music = notes_array[ind][:time_steps]
random_music = random_music.reshape(1,time_steps)

In [ ]:
prob  = model.predict(random_music)[0]
if np.random.random(1)>0.15:
  y_pred= np.argmax(prob, axis = 0)
  print(y_pred)
else:
  t = np.random.randint(0,5)*-1
  y_pred = np.argsort(prob, axis=0)[t]

ValueError: ignored

array([ 20,  18, 248,  13, 237, 222, 298,  11, 273, 247,  25,  85, 264,
       242, 234, 249, 233, 204, 200, 238, 223, 164, 228, 206,  65, 256,
       130, 259, 192, 140, 291, 284, 269, 178, 267, 170,  66, 121, 186,
       281,  84, 245, 207, 250, 227, 236, 293, 287,  42, 199, 197, 272,
       231,  64, 105,  30, 277, 165,  82, 290, 253,  62, 194,  96,  68,
        15, 175,  83, 258, 271,  27, 240,   9,  69, 113,  79, 244, 262,
       183,  99,  46, 103, 109, 188, 191,   7, 179, 243, 123, 120, 129,
       128, 125, 226,   5, 265, 119,  87, 297, 286,  97, 182, 216, 162,
       101, 100, 159, 111, 252, 221, 212, 219, 255, 292, 201, 126, 232,
       229, 268, 257, 117,  52,   6, 107,  81,  95, 132, 167,  56,  49,
        58, 266,   0,  50, 104, 180,  89,  74,   3, 235,  63,  61,  47,
        70, 143,  94,   1,  76, 294,  88,  44, 134, 131,  51, 220,  90,
        93, 133,  80,  67, 136,  59, 135, 230,  91,  10,  86, 144, 106,
        92,  77,  60,  72,  73,   8,  23,   2, 127, 283, 118, 24

In [ ]:
random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
random_music = random_music[1:]

In [ ]:
random_music

array([[138,  29, 160,  33, 145,  36, 141,  38, 155,  33, 143,  36, 146,
         38, 142,  33, 154,  36, 144,  38, 299, 141,  36, 169,  33, 146,
         36, 142,  38, 160,  38, 145]])